In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [34]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-04 05:00:00+00:00


In [35]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [36]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-03 23:07:31,736 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 23:07:31,809 INFO: Initializing external client
2025-03-03 23:07:31,811 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 23:07:33,539 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672


In [37]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.86s) 


In [38]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,150,0.0,2025-03-04 00:00:00+00:00
1,131,0.0,2025-03-04 00:00:00+00:00
2,74,8.0,2025-03-04 00:00:00+00:00
3,253,0.0,2025-03-04 00:00:00+00:00
4,201,0.0,2025-03-04 00:00:00+00:00
...,...,...,...
497,117,0.0,2025-03-04 05:00:00+00:00
498,73,0.0,2025-03-04 05:00:00+00:00
499,174,0.0,2025-03-04 05:00:00+00:00
500,258,0.0,2025-03-04 05:00:00+00:00


In [39]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-03 23:08:18,068 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 23:08:18,077 INFO: Initializing external client
2025-03-03 23:08:18,077 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 23:08:19,735 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672
Current UTC time: 2025-03-04 04:08:18.068859+00:00
Next hour: 2025-03-04 05:00:00+00:00
Querying for date range: 2025-03-04 to 2025-03-05
Filtering for hour: 2025-03-04 05:00
Found 251 records


In [40]:
now = datetime.now(timezone.utc)

In [41]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
251,120,0.0,2025-03-04 05:00:00+00:00
252,61,1.0,2025-03-04 05:00:00+00:00
253,90,70.0,2025-03-04 05:00:00+00:00
254,33,3.0,2025-03-04 05:00:00+00:00
255,32,0.0,2025-03-04 05:00:00+00:00
...,...,...,...
497,117,0.0,2025-03-04 05:00:00+00:00
498,73,0.0,2025-03-04 05:00:00+00:00
499,174,0.0,2025-03-04 05:00:00+00:00
500,258,0.0,2025-03-04 05:00:00+00:00


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  502 non-null    int32                  
 1   predicted_demand    502 non-null    float64                
 2   pickup_hour         502 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 9.9 KB


In [43]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [44]:
dt = current_date.ceil('h')

In [45]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-03 23:10:20,730 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 23:10:20,746 INFO: Initializing external client
2025-03-03 23:10:20,746 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 23:10:22,356 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.62s) 


In [47]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.40s) 


,pickup_location_id,predicted_demand,pickup_hour


In [48]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,150,0.0,2025-03-04 00:00:00+00:00
1,131,0.0,2025-03-04 00:00:00+00:00
2,74,8.0,2025-03-04 00:00:00+00:00
3,253,0.0,2025-03-04 00:00:00+00:00
4,201,0.0,2025-03-04 00:00:00+00:00
...,...,...,...
497,117,0.0,2025-03-04 05:00:00+00:00
498,73,0.0,2025-03-04 05:00:00+00:00
499,174,0.0,2025-03-04 05:00:00+00:00
500,258,0.0,2025-03-04 05:00:00+00:00


In [49]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [50]:
predictions = fetch_next_hour_predictions()

2025-03-03 23:15:55,396 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 23:15:55,413 INFO: Initializing external client
2025-03-03 23:15:55,413 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 23:15:57,166 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672
Current UTC time: 2025-03-04 04:15:55.396420+00:00
Next hour: 2025-03-04 05:00:00+00:00
Found 251 records


In [51]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

2025-03-03 23:16:03,574 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 23:16:03,590 INFO: Initializing external client
2025-03-03 23:16:03,590 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 23:16:05,353 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672


In [52]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.43s) 


In [53]:
df["pickup_hour"].max()

Timestamp('2025-03-04 05:00:00+0000', tz='Etc/UTC')

In [54]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-04 05:00:00+00:00


In [55]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
251,120,0.0,2025-03-04 05:00:00+00:00
252,61,1.0,2025-03-04 05:00:00+00:00
253,90,70.0,2025-03-04 05:00:00+00:00
254,33,3.0,2025-03-04 05:00:00+00:00
255,32,0.0,2025-03-04 05:00:00+00:00
...,...,...,...
497,117,0.0,2025-03-04 05:00:00+00:00
498,73,0.0,2025-03-04 05:00:00+00:00
499,174,0.0,2025-03-04 05:00:00+00:00
500,258,0.0,2025-03-04 05:00:00+00:00
